In [2]:
import random

# Define Personas and their Purchase Habits
personas = {
    "The Morning Commuter": {"percentage": 20, "purchases": {"Espresso Drinks": 1, "Pour Over Coffee": 0.5, "Pastries": 1, "Sandwiches": 0.5}},
    "The Student": {"percentage": 15, "purchases": {"Bubble Tea": 1, "Bubble Tea With Upgrades": 0.5, "Cookies": 1, "Specialty Snacks": 0.5, "Hot Chocolate": 0.5}},
    "The Remote Worker": {"percentage": 15, "purchases": {"Pour Over Coffee": 1, "Espresso Drinks": 1, "Sparkling Tea Drinks": 0.5, "Pastries": 1, "Sandwiches": 1}},
    "The Socializer": {"percentage": 10, "purchases": {"Specialty Snacks": 1, "Bubble Tea With Upgrades": 1, "Sparkling Tea Drinks": 0.5, "Tea": 0.5, "Cookies": 1, "Pie Slices": 0.5}},
    "The Health-Conscious Customer": {"percentage": 5, "purchases": {"Tea": 1, "Sparkling Tea Drinks": 1, "Specialty Snacks (healthy options)": 1, "Sandwiches (healthy fillings)": 1}},
    "The Family Outing": {"percentage": 15, "purchases": {"Kids Drinks": 2, "Hot Chocolate": 1, "Cookies": 2, "Pastries": 2, "Sandwiches": 2}},
    "The Connoisseur": {"percentage": 10, "purchases": {"Pour Over Coffee": 1, "Espresso Drinks": 1, "Specialty Snacks": 1, "Pie Slices": 1}},
    "The Professional Freelancer": {"percentage": 10, "purchases": {"Espresso Drinks": 1.5, "Pour Over Coffee": 0.5, "Tea": 0.5, "Sparkling Tea Drinks": 0.5, "Bubble Tea": 0.2, "Pastries": 1, "Cookies": 0.5, "Sandwiches": 1, "Specialty Snacks": 0.3}},
}

# New: Define average prices for each menu item
prices = {
    "Espresso Drinks": 5.0,
    "Pour Over Coffee": 5.0,
    "Pastries": 5,
    "Sandwiches": 7.0,
    "Bubble Tea": 5.5,
    "Bubble Tea With Upgrades": 6,
    "Cookies": 4.0,
    "Specialty Snacks": 5.0,
    "Hot Chocolate": 5.0,
    "Tea": 5.0,
    "Sparkling Tea Drinks": 5.0,
    "Specialty Snacks (healthy options)": 5.5,
    "Sandwiches (healthy fillings)": 7.0,
    "Kids Drinks": 4.0,
    "Pie Slices": 5.0,
}

item_relations = {
    "Espresso Drinks": {"Pastries": 0.2, "Cookies": 0.15},
    "Pour Over Coffee": {"Pastries": 0.2, "Pie Slices": 0.1},
    "Pastries": {"Espresso Drinks": 0.1, "Pour Over Coffee": 0.1, "Tea": 0.05},
    "Sandwiches": {"Espresso Drinks": 0.05, "Sparkling Tea Drinks": 0.1},
    "Bubble Tea": {"Specialty Snacks": 0.1},
    "Bubble Tea With Upgrades": {"Specialty Snacks": 0.15, "Cookies": 0.1},
    "Cookies": {"Espresso Drinks": 0.15, "Tea": 0.1},
    "Specialty Snacks": {"Bubble Tea": 0.1, "Sparkling Tea Drinks": 0.1},
    "Hot Chocolate": {"Cookies": 0.2, "Pastries": 0.15},
    "Tea": {"Pastries": 0.1, "Cookies": 0.1, "Pie Slices": 0.1},
    "Sparkling Tea Drinks": {"Specialty Snacks": 0.1, "Sandwiches": 0.05},
    "Specialty Snacks (healthy options)": {"Tea": 0.1, "Sparkling Tea Drinks": 0.1},
    "Sandwiches (healthy fillings)": {"Tea": 0.05, "Sparkling Tea Drinks": 0.05},
    "Kids Drinks": {"Cookies": 0.2, "Pastries": 0.1},
    "Pie Slices": {"Pour Over Coffee": 0.1, "Tea": 0.1},
}

def adjust_probability(item, selected_items):
    adjustment = 0
    for related_item, effect in item_relations.get(item, {}).items():
        if related_item in selected_items:
            adjustment += effect
    return adjustment

def calculate_purchases_and_revenue(estimated_total_customers):
    total_purchases = {item: 0 for item in prices.keys()}
    total_revenue = {item: 0 for item in prices.keys()}
    
    for persona, data in personas.items():
        num_customers_in_persona = (data['percentage'] / 100) * estimated_total_customers
        
        for _ in range(int(num_customers_in_persona)):
            selected_items = []  # Track selected items for this customer
            for item, probability in data['purchases'].items():
                # Adjust probability based on previously selected items
                adjusted_probability = probability + adjust_probability(item, selected_items)
                if random.random() < adjusted_probability:
                    total_purchases[item] += 1
                    total_revenue[item] += prices[item]
                    selected_items.append(item)
    
    total_purchases_sum = sum(total_purchases.values())
    total_revenue_sum = sum(total_revenue.values())
    
    return total_purchases, total_revenue, total_purchases_sum, total_revenue_sum

def calculate_purchases_and_revenue_monte_carlo(estimated_total_customers, num_simulations):
    # Initialize accumulators for total purchases and revenue for each item
    item_purchases_sums = {item: 0 for item in prices.keys()}
    simulations_revenue = []
    
    for i in range(num_simulations):
        total_purchases, total_revenue, _, total_revenue_sum = calculate_purchases_and_revenue(estimated_total_customers)
        
        # Accumulate total purchases and revenue for each item
        for item in total_purchases.keys():
            item_purchases_sums[item] += total_purchases[item]
        simulations_revenue.append(total_revenue_sum)
        
        # Log completion of each simulation
        if (i+1) % 1000 == 0 or i == num_simulations - 1:  # Log at intervals to reduce console clutter
            print(f"Simulation {i+1}/{num_simulations} completed.")
    
    # Calculate the average total purchases and revenue across all simulations
    avg_item_purchases = {item: total / num_simulations for item, total in item_purchases_sums.items()}
    avg_total_revenue = sum(simulations_revenue) / num_simulations
    
    # The total sum of purchases across all items is not directly meaningful in the context of averages per item,
    # so we calculate the average revenue and the average purchases per item instead.
    return avg_item_purchases, avg_total_revenue

estimated_total_customers = 2000
num_simulations = 5000

# Run the Monte Carlo simulation with the updated function
avg_item_purchases, avg_total_revenue = calculate_purchases_and_revenue_monte_carlo(estimated_total_customers, num_simulations)

# Displaying the average total purchases for each menu item
for item, avg_purchases in avg_item_purchases.items():
    print(f"{item}: Average Purchases = {avg_purchases:.2f}")

# Displaying the average total revenue across all items

# Displaying the average results
print(f"\nAverage Total Revenue Across All Items: ${avg_total_revenue:.2f}")


Simulation 1000/5000 completed.
Simulation 2000/5000 completed.
Simulation 3000/5000 completed.
Simulation 4000/5000 completed.
Simulation 5000/5000 completed.
Espresso Drinks: Average Purchases = 1100.00
Pour Over Coffee: Average Purchases = 799.84
Pastries: Average Purchases = 1200.00
Sandwiches: Average Purchases = 1019.83
Bubble Tea: Average Purchases = 339.89
Bubble Tea With Upgrades: Average Purchases = 349.77
Cookies: Average Purchases = 940.14
Specialty Snacks: Average Purchases = 654.20
Hot Chocolate: Average Purchases = 509.93
Tea: Average Purchases = 299.89
Sparkling Tea Drinks: Average Purchases = 469.96
Specialty Snacks (healthy options): Average Purchases = 100.00
Sandwiches (healthy fillings): Average Purchases = 100.00
Kids Drinks: Average Purchases = 300.00
Pie Slices: Average Purchases = 309.91

Average Total Revenue Across All Items: $44036.05
